In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
contract=pd.read_csv("../input/categories/contact.csv")

In [3]:
contract.head()

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-004567394_00861,Complete,https://transcription.si.edu/transcribe/11400/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11400/image/NMAAHC-004567394_00861.jpg,1,[D104 ENCL]\r\nNorth Carolina\r\nThis indentur...,/11400/transcriptions/NMAAHC-004567394_00861.txt,\N,\N,11400,Contracts,NaN,NaN
1,NMAAHC-004567394_00957,Complete,https://transcription.si.edu/transcribe/11400/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11400/image/NMAAHC-004567394_00957.jpg,1,[E 11 ENCL] \r\n\r\n\r\nState of North Carolin...,/11400/transcriptions/NMAAHC-004567394_00957.txt,\N,\N,11400,Contracts,NaN,NaN
2,NMAAHC-004567394_00958,Complete,https://transcription.si.edu/transcribe/11400/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11400/image/NMAAHC-004567394_00958.jpg,1,"near Lizzie Etheridge's gate, thence South 56 ...",/11400/transcriptions/NMAAHC-004567394_00958.txt,\N,\N,11400,Contracts,NaN,NaN
3,NMAAHC-004567395_00439,Complete,https://transcription.si.edu/transcribe/11406/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11406/image/NMAAHC-004567395_00439.jpg,1,Copy\r\n\r\nAgreement of Apprenticeship\r\nBu ...,/11406/transcriptions/NMAAHC-004567395_00439.txt,\N,\N,11406,Contracts,NaN,NaN
4,NMAAHC-004567395_00440,Complete,https://transcription.si.edu/transcribe/11406/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11406/image/NMAAHC-004567395_00440.jpg,1,[H 132 ENCL] \r\n\r\nCopy\r\n\r\nAgreement of ...,/11406/transcriptions/NMAAHC-004567395_00440.txt,\N,\N,11406,Contracts,NaN,NaN


In [4]:
contract.shape

(23731, 15)

In [5]:
contract = contract[pd.notnull(contract['transcription_text'])]
contract = contract.reset_index(drop=True)

In [6]:
contract.isna().sum()

document_id                             0
status                                  0
document_url                            0
media_type                              0
media_link                              0
relative_media_path                     0
is_media_downloaded                     0
transcription_text                      0
transcription_text_relative_path        0
from_time                               0
to_time                                 0
project_id                              0
Category                                0
Unnamed: 13                         23729
Unnamed: 14                         23729
dtype: int64

In [7]:
contract_text=contract["transcription_text"]

In [8]:
contract_text.shape

(23729,)

In [9]:
print(contract_text[12540])

Know all Men by These Presents, That I, M.C. Abbey of the County of Yellowbusha State of Mississippi held and firmly bound to the United States of America in the sum of Four hundred & thirty two Dollars, for the payment of which I bind myself heirs, executors and administrators, firmly by these presents, in this Contract: That I am to furnish the persons, whose names are subjoined, freed laborers, Quarters, Fuel, substantial and healthy Rations, [[strikethrough]] all necessary Medical Attendance and Supplies in case of sickness [[/strikethrough]], and the amount set opposite their respective names per year, during the continuation of this Contract—the laborers to be paid in full before the final disposal of the crop which is to be raised by them on my plantation, in the County of Yellowbusha State of Mississippi.

[[8 column table]]
| NO. | NAMES. | AGE. | Rate of pay per month | NO. | NAMES. | AGE. | Rate of pay per month |
| --- | --- | --- | --- | --- | --- | --- | --- |

| 1 | Gree

In [10]:
print(contract_text[15000])

Original 

AGREEMENT FOR LABOR.

THIS INDENTURE, Made and entered into on the 1st day of January A. D, 1867, between Thomas D. Chunn of the County of Woodruff and State of [[strikethrough]] North Carolina [[/strikethrough]] Arkansas, of the first part, and the undersigned, Free Laborer, of the second part: 

WITNESSETH, That the said Laborers have agreed to work for the said Chunn 12 months from this date to the 31 day of December 1867; to do all kinds of work common to farms of the country; to do their work faithfully, and to be respectful in their deportment. And they further agree, that they only receive every month, the amount placed opposite their names, and the rest at the end of the year; and that the amount so retained shall be forfeited if a violation of this contract is clearly proven. And they further agree, that time lost by idleness or absence without leave, shall not be paid for. 

And the said Thomas D. Chunn has agreed that he will furnish the said laborers with comfort

In [11]:
print(contract_text[500])

Contract for labor
indenture made and enterered into this the 22nd day of Sept 1865 between Duncan McNair of the County of Robeson and State of North Carolina of the one part and the undersigned freed laborers of the the second part. Witnesseth that the said laborers have agreed from this date to the first day of January 1866 to do all kinds of labour common to the farms of the County to work ten hours per day, to do their work faithfully, and to be respectful in their deportment, and they further agree that one half of their pay shall be retained by the said Duncan McNair till the end of the year, and the amt so retained shall be forfeited by a violation of this contract, & they further agree that the time lost by idleness or absence without leave, shall not be paid for, & the said Duncan McNair has agreed that he will furnish the said laborers with comfortable quarters, sufficient  rations and the amt of money per month, which hands affixed their respective names that he will treat t

In [12]:
print(contract_text[1000])

[[preprinted]]
AGREEMENT FOR LABOR.

THIS INDENTURE, Made and entered into on the ^[[12]] day of ^[[Feby]] A.D, 186^[[7]], between ^[[Henry Pearce]] of the County of ^[[Franklin]] and State of North Carolina, of the first part, and the under-signed, Free Laborer, of the second part: ^[[Kinchen Gupton]]

WITNESSETH, That the said Laborers have agreed to work for the said ^[[Henry Pearce]] from this date to the ^[[1st]] day of ^[[August [[strikethrough]] 186 [[/strikethrough]]]] 186^[[7]]; to do all kinds of work common to farms of the country; to do their work faith-fully, and to be respectful in their deportment. And they further agree, that they only receive every month, the amount placed opposite their names, and the rest at the end of the year; and that the amount so retained shall be forfeited if a violation of this contract is clearly proven. And they further agree, that time lost by idleness or absence without leave, shall not be paid for.

And the said ^[[Henry Pearce]] has agre

In [13]:
print(contract_text[100])

Brunswick Co NC May 13/65

Wm Waters & Freedmen (24)

Agreement to cultivate crops at Clarendon

23 men — 10 women 23 children


In [14]:
contract_text[3511]

'Joseph Parker\r\nJune 4th 1865\r\n\r\nJoseph Parker\r\nPaid'

In [15]:
len(contract_text)

23729

# Preprocessing

In [16]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    return s


In [17]:
start=time.time()
for i in tqdm(range(len(contract_text))):
    contract_text[i] = process(contract_text[i])
for i in tqdm(range(len(contract_text))):
    contract_text[i] = " ".join(contract_text[i])
end=time.time()
print(end-start)

100%|██████████| 23729/23729 [00:12<00:00, 1977.40it/s]

69.87494087219238


In [18]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 38.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

In [19]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-28 10:53:10,392 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-28 10:53:39,271 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [20]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-28 10:53:51,614 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-28 10:53:53,781 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [21]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-28 10:54:52,114 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-28 10:55:11,086 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


# Ontonotes

In [22]:
lst=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
LOC=[]
Bulding=[]
Cardinal=[]
from flair.data import Sentence

contract_onto=pd.DataFrame({"contract_text":"", "Name-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(23729):
        sentencess =Sentence(contract_text[i])
        contract_onto.loc[i,"contract_text"]=contract_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        L=[]
        O=[]
        B=[]
        C=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.97) & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
                elif dc[j]=="FAC":
                      B.append(j)
                elif dc[j]=="CARDINAL":
                      C.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
        LOC.append(L)
        Bulding.append(B)
        Cardinal.append(C)
        
            
contract_onto["Date-ontonotes"]=Date
contract_onto["Name-ontonotes"]=Per
contract_onto["GPE-ontonotes"]=GPE
contract_onto["ORG-ontonotes"]=ORG 
contract_onto["LOC-ontonotes"]=LOC 
contract_onto["Building-ontonotes"]=Bulding
contract_onto["Cardinal-ontonotes"]=Cardinal

2022-06-28 10:57:38,594 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 11:02:47,664 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 11:10:30,586 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 11:10:34,382 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 11:11:43,935 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 11:11:47,054 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [23]:
contract_onto["contract_text"]=contract_text

In [24]:
contract_onto.head()

,contract_text,Name-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes,Cardinal-ontonotes
0,North Carolina This indenture made entered Thi...,"[H.G . Cutler, Thos R Underwood, Alfred A McKe...","[North Carolina, County of Carteret]",[],[This 5th day of November 1853],"[Stake North, Bogue Sound, Shepperd Creek]",[],"[five hundred ten, 13.0, 203, 209, 4, 85, 116,..."
1,State of North Carolina Currituck County . Thi...,"[John B. Etheridge, Little John Etheridge, Joh...","[North Carolina, Currituck County, County of C...",[],"[fifth day of September , year, One Thousand e...","[Roanoke Island, Croatain Sound]",[],[]
2,"near Lizzie Etheridges gate , thence South 56 ...","[Lizzie Etheridges, Little John Etheridge, Joh...",[],[],[],"[Red Hill, Croatan Sound, Sound]",[],[]
3,Copy Agreement of Apprenticeship Bu Mrs. Kate ...,"[Kate V.Chamblin, James Samenett, Betty Taylor...",[Carroll Parish La],[U.S. Army],"[Oct. 1 1866, Nov. 3 1866]",[],[],[]
4,Copy Agreement of Apprenticeship . This agreem...,"[Kate V. Chambliss, James Samenett, Betty Tayl...",[New Orleans Louisiana],[],"[1 day of October A . D. 1866, Octr 31 1865, 1...",[],[],[two]


In [25]:
contract_onto["Date-ontonotes"]

0                          [This 5th day of November 1853]
1        [fifth day of September , year, One Thousand e...
2                                                       []
3                               [Oct. 1 1866, Nov. 3 1866]
4        [1 day of October A . D. 1866, Octr 31 1865, 1...
                               ...                        
23724                                                   []
23725                                                   []
23726                                                   []
23727                                                   []
23728                                                   []
Name: Date-ontonotes, Length: 23729, dtype: object

In [26]:
G=[]
for i in contract_onto["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-Z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)
    
            
      

In [27]:
len(GF)

23729

In [28]:
contract_onto["postDate-ontonotes"]=GF

In [29]:
G=[]
for i in contract_onto["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

contract_onto["postDate-ontonotes"]=G

In [30]:
NG=[]
for i in G:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if date !=[]:
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST","Augst","Augt","AUG","Augusta"]:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER",'Sepr','Septh']:
                    dates[1]= "Septembre"
                elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                    dates[1]= "October"
                elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER", "The"]:
                    dates[1]= "November"
                elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                    dates[1]= "December"
                elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                    dates[1]= "January"
                elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                    dates[1]= "February"
                elif dates[1] in ["March","Mar"]:
                    dates[1]= "March"
                elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                    dates[1]= "April"
                elif dates[1] in ["May","MAY"]:
                    dates[1]= "May"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]= "June"
                elif dates[1] in ["JULY","July","JUL"]:
                    dates[1]= "July"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
                date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j ) 
                dates=[]
                for k in date:
                    for l in k:
                        dates.append(l)
                
                
                    if dates[1] in ["August","Aug","augt","AUGUST","Augst",'Augt',"AUG","Augusta"]:
                        dates[1]= "August"
                    elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER","Sepr","Septh", "This", "This "]:
                        dates[1]= "Septembre"
                    elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                        dates[1]= "October"
                    elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                        dates[1]= "November"
                    elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                        dates[1]= "December"
                    elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                        dates[1]= "January"
                    elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                        dates[1]= "February"
                    elif dates[1] in ["March","Mar"]:
                        dates[1]= "March"
                    elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                        dates[1]= "April"
                    elif dates[1] in ["May","MAY"]:
                        dates[1]= "May"
                    elif dates[1] in ["JUN","Jun","June"]:
                        dates[1]= "June"
                    elif dates[1] in ["JULY","July","JUL"]:
                        dates[1]= "July"
                    if dates[0]== "fifteenth" :
                        dates[0]="15"
                    elif (dates[0]=="sixteenth") |(dates[0]=="This 16 "):
                        dates[0]="16"
                    elif dates[0]=="Twenty":
                        dates[0]="20" 
                    elif dates[0]=="Twentyseventh":
                        dates[0]="27" 
                    elif (dates[0]=="Twenty Eighth") |(dates[0]=="TwentyEight") |(dates[0]=="TwentyEight ") |(dates[0]=="Twentyeight"):
                        dates[0]="28"
                    elif (dates[0]=="Ninth") |(dates[0]=="Ninth ")|(dates[0]=="Augusta 9th ") |(dates[0]=="ninth") |(dates[0]=="ninth ")|(dates[0]=="Augusta 9th"):
                        dates[0]="9"
                    
                    elif dates[0]=='Seventh':
                        dates[0]="7" 
                    elif (dates[0]=='The') |(dates[0]=="The 9th") :
                        dates[0]="9"
                    elif dates[0]=="the 9" :
                        dates[0]="9"
                    elif (dates[0]=="Eighth ") |(dates[0]=="Eighth") |(dates[0]=="eighth") |(dates[0]=="eighth "):
                        dates[0]="8"
                    elif dates[0]=="Seventh ":
                        dates[0]="7"
                    elif (dates[0]=="Twenty ninth ")| (dates[0]=="Twenty ninth") | (dates[0]=="twenty ninth ")|(dates[0]=="Twenty Ninth ")|(dates[0]=="Twenty Ninth") :
                        dates[0]="29"
                    elif (dates[0]=="Thirty") |(dates[0]=="Thirty ") | (dates[0]=="thirtieth "):
                        dates[0]="30"
                    elif (dates[0]=="Thirty first ") |(dates[0]=="Thirty first"):
                        dates[0]="31"
                  
                    
                    elif dates[0]=="first ":
                        dates[0]="1"
                    elif (dates[0]=="Second ") |(dates[0]=="Second") |(dates[0]=="second")|(dates[0]=="second "):
                        dates[0]="2"
                    elif dates[0]=="fourth ":
                        dates[0]="4"
                    elif dates[0]=="tenth ":
                        dates[0]="10"
                    elif (dates[0]=="Fifth ")|(dates[0]=="Fifth") |(dates[0]=="fifth ")|(dates[0]=="fifth"):
                        dates[0]="5"
                    elif (dates[0]=="This") | (dates[0]=="This 7th ")|(dates[0]=="seventh ") |(dates[0]=="seventh") |(dates[0]=="Seventh "):
                        dates[0]="7"
                    elif (dates[0]=="This 31st") |(dates[0]=="Thirty-first "):
                        dates[0]="31"
                    elif (dates[0]=="Thirtieth") |(dates[0]=="Thirtieth ") |(dates[0]=="Thirty"):
                        dates[0]="30"
                    elif dates[0]=="twenty fifth ":
                        dates[0]="25"
                    elif dates[0]=="Twenty seventh ":
                        dates[0]="27"
                    elif (dates[0]=="Twenty sixth ") |(dates[0]=="Twentysixth ")|(dates[0]=="Twentysixth") |(dates[0]=="twentysixth "):
                        dates[0]="26"
                    elif dates[0]=="third ":
                        dates[0]="3" 
                    elif dates[0]=="third":
                        dates[0]="3"
                    elif dates[0]=="Third ":
                        dates[0]="3"
                    elif dates[0]=="Third":
                        dates[0]="3"
                    
                    elif dates[0]=="Tenth":
                        dates[0]="10" 
                    elif dates[0]=="Augusta":
                        dates[0]="9" 
                    elif (dates[0]=="this_sixth ")|(dates[0]=="sixth ")| (dates[0]=="Sixth") | (dates[0]=="Sixth ") |(dates[0]=="this sixth "):
                        dates[0]="6" 
                        
                    elif dates[0]=="Tenth ":
                        dates[0]="10"
                    elif (dates[0]=="twentieth")|(dates[0]=="This 20th "):
                        dates[0]="20"
                    elif (dates[0]=="twenty fourth ") |(dates[0]=="twenty fourth") |(dates[0]=="Twenty fourth ") |(dates[0]=="Twenty fourth"):
                        dates[0]="24"
                    elif (dates[0]=="fifteenth ") |(dates[0]=="Fifteenth ") |(dates[0]=="Fifteenth"):
                        dates[0]="15"
                    elif dates[0]=="twentieth ":
                        dates[0]="20"
                    elif dates[0]=="Eighteenth ":
                        dates[0]="18"
                    elif dates[0]=="Eighteenth":
                        dates[0]="18"
                    elif (dates[0]=="Nineteenth ") |(dates[0]=="Nineteenth"):
                        dates[0]="19"
                    elif dates[0]=="First ":
                        dates[0]="1"
                    elif dates[0]=="First":
                        dates[0]="1"
                    elif (dates[0]=="Sixteenth ")|(dates[0]=="this 16") |(dates[0]=="Sixteenth"):
                        dates[0]="16"
                   
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif dates[0]=="Twenty third ":
                        dates[0]="23"
                    elif dates[0]=="Twenty Second ":
                        dates[0]="22"
                    elif dates[0]=="Twenty Second":
                        dates[0]="22"
                    elif dates[0]=="Twenty Eighth ":
                        dates[0]="28"
                    elif dates[0]=="Twentysenventh ":
                        dates[0]="27"
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif (dates[0]=="twenty third") |(dates[0]=="Twentythird")|(dates[0]=="Twentythird "):
                        dates[0]="23"
                    elif (dates[0]=="fourteenth ") | (dates[0]=="fourteenth") | (dates[0]=="Fourteenth") | (dates[0]=="Fourteenth "):
                        dates[0]="14"
                    elif (dates[0]=="Eleventh ") | (dates[0]=="Eleventh") |(dates[0]=="eleventh ") :
                        dates[0]="11"
                    elif (dates[0]=="Thirteenth") | (dates[0]=="Thirteenth ") :
                        dates[0]="13"
                    elif (dates[0]=="seventeenth ") |(dates[0]=="seventeenth") |(dates[0]=="Seventeenth ") :
                        dates[0]="17"
                     
                     
                     
                    
                        
                    j = dates[0]+"-"+dates[1]+"-"+dates[2]
                    ng.append(j)
            
            
    NG.append(ng)   

In [31]:
contract_onto["standard_date-ontonotes"]=NG

In [32]:
contract_onto.to_csv("contract_onto.csv",index=False)

# Ner english large

In [33]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
Other_name=[]
from flair.data import Sentence

contract_large=pd.DataFrame({"contract_text":"", "Name-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(23729):
        sentencess =Sentence(contract_text[i])
        contract_large.loc[i,"contract_text"]=contract_text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        Other=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") :
                      O.append(j)
                elif  (dc[j]=="MISC") :
                      Other.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
        Other_name.append(Other)
            

contract_large["Name-nerenglarge"]=Per
contract_large["LOC-nerenglarge"]=LOC
contract_large["ORG-nerenglarge"]=ORG    
contract_large["other_namelarge"] =Other_name 

2022-06-28 11:21:37,750 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 11:26:46,208 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 11:34:28,843 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 11:34:32,463 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 11:35:40,924 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 11:35:44,034 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [34]:
contract_large.head()

,contract_text,Name-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge
0,North Carolina This indenture made entered Thi...,"[H.G . Cutler, Thos R Underwood, Alfred A McKe...","[North Carolina, County of Carteret, Bogue Sou...",[],[]
1,State of North Carolina Currituck County . Thi...,"[John B. Etheridge, Little John Etheridge]","[State of North Carolina, Currituck County, Co...",[],[]
2,"near Lizzie Etheridges gate , thence South 56 ...","[Little John Etheridge, John B Etheridge, A Do...","[Lizzie Etheridges gate, Red Hill, Croatan Sou...",[],[]
3,Copy Agreement of Apprenticeship Bu Mrs. Kate ...,"[Kate V.Chamblin, 1st, Geo . W. Rollins, Mary ...","[La, Lake Providence Carroll]",[U.S. Army],[]
4,Copy Agreement of Apprenticeship . This agreem...,"[Mrs, Kate V. Chambliss, James Samenett, Betty...","[Carroll Parish, New Orleans, Louisiana, Freed...",[],[African]


In [35]:
contract_large.to_csv("contract_large.csv",index=False)

# Ner english


In [36]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
Other_name=[]
from flair.data import Sentence

contract_ner=pd.DataFrame({"contract_text":"", "PERSON-nerenglish":"","LOC-nerenglish":"","ORG-nerenglish":[]})
for i in range(23729):
        sentencess =Sentence(contract_text[i])
        contract_ner.loc[i,"contract_text"]=contract_text[i]
        tagger1.predict(sentencess)
        dc={}
        P=[]
        L=[]
        O=[]
        Other=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG")  :
                      O.append(j)
                elif  (dc[j]=="MISC") :
                      Other.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
        Other_name.append(Other)
            

contract_ner["Name-nerenglish"]=Per
contract_ner["LOC-nerenglish"]=LOC
contract_ner["ORG-nerenglish"]=ORG 
contract_ner["Other_name-nerenglish"]=Other_name 

2022-06-28 11:51:37,220 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 12:10:56,639 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 12:39:29,439 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 12:39:42,069 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 12:43:48,730 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 12:44:00,637 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [37]:
contract_ner.head()

,contract_text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Name-nerenglish,Other_name-nerenglish
0,North Carolina This indenture made entered Thi...,NaN,"[County of Carteret, Shepperd Creek, Creek Sou...",[],"[Thos R Underwood, Alfred A McKerhan, Wm H Was...",[]
1,State of North Carolina Currituck County . Thi...,NaN,"[County State, Witnesseth, County of Currituck...",[],[John B. Etheridge],[]
2,"near Lizzie Etheridges gate , thence South 56 ...",NaN,"[Red Hill, Croatan Sound]",[],"[John B Etheridge, Douglas Adam D. Etheridge]",[]
3,Copy Agreement of Apprenticeship Bu Mrs. Kate ...,NaN,[],[],[Kate V.Chamblin],[]
4,Copy Agreement of Apprenticeship . This agreem...,NaN,[],[],"[Kate V. Chambliss, Betty Taylor Mary Taylor, ...",[African]


In [38]:
contract_ner.to_csv("contract_ner.csv",index=False)

In [39]:
contract_ner["Name-nerenglarge"]=contract_large["Name-nerenglarge"]
contract_ner["LOC-nerenglarge"]=contract_large["LOC-nerenglarge"]
contract_ner["ORG-nerenglarge"]=contract_large["ORG-nerenglarge"]
contract_ner["other_namelarge"]=contract_large["other_namelarge"]
contract_ner["PERSON-nerenglish"]=contract_ner["Name-nerenglish"]
contract_ner["LOC-nerenglish"]=contract_ner["LOC-nerenglish"]
contract_ner["ORG-nerenglish"]=contract_ner["ORG-nerenglish"]
contract_ner["Other_name-nerenglish"]=contract_ner["Other_name-nerenglish"]

In [40]:
to_compare_result_contract_ner=contract_ner

In [41]:
to_compare_result_contract_ner.to_csv("to_compare_result_contract_ner.csv", index=False)